<a href="https://colab.research.google.com/github/rlagusgh0223/ICT-2/blob/main/220410_1(kowiki).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **[환경설정]**

런타임 - 런타임 유형 변경 - GPU 설정 후  
도구 - 설정 - 편집기 - 행 번호 표시 하면 행 번호 나온다

▷ install

In [56]:
!pip install sentencepiece

▷ import

In [57]:
import os
import random
import shutil
import json
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K
from tqdm.notebook import tqdm

In [58]:
# random seed initialize
random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [59]:
# GPU 사용 확인
!nvidia-smi

Sun Apr 10 03:01:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [64]:
# google drive mount
# content에 dirve폴더가 없으니까 만든 다음에 실행
# 폴더가 꼬인것 같으니까 난 sdrive로 할 것
from google.colab import drive
drive.mount('/content/sdrive', force_remount=True)

Mounted at /content/sdrive


In [65]:
# data dir
data_dir = '/content/sdrive/MyDrive/Colab Notebooks/Data/nlp'
os.listdir(data_dir)

[]

## **[Vocabulary]**
▷ 한국어 위키 다운로드

In [66]:
# korean wiki dir
kowiki_dir = os.path.join(data_dir, 'kowiki')
if not os.path.exists(kowiki_dir):
 os.makedirs(kowiki_dir)
os.listdir(kowiki_dir)

[]

In [67]:
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-meta-current.xml.bz2

--2022-04-10 03:06:40--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-meta-current.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1031063543 (983M) [application/octet-stream]
Saving to: ‘kowiki-latest-pages-meta-current.xml.bz2’

kowiki-latest-pages 100%[===================>] 983.30M  4.93MB/s    in 3m 21s  

2022-04-10 03:10:01 (4.88 MB/s) - ‘kowiki-latest-pages-meta-current.xml.bz2’ saved [1031063543/1031063543]



In [68]:
# WikiExtractor 다운로드
!wget https://github.com/paul-hyun/web-crawler/raw/master/WikiExtractor.py

--2022-04-10 03:10:34--  https://github.com/paul-hyun/web-crawler/raw/master/WikiExtractor.py
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/paul-hyun/web-crawler/master/WikiExtractor.py [following]
--2022-04-10 03:10:34--  https://raw.githubusercontent.com/paul-hyun/web-crawler/master/WikiExtractor.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119222 (116K) [text/plain]
Saving to: ‘WikiExtractor.py’

WikiExtractor.py    100%[===================>] 116.43K  --.-KB/s    in 0.02s   

2022-04-10 03:10:34 (5.74 MB/s) - ‘WikiExtractor.py’ saved [119222/119222]



In [70]:
# 파일이 잘 받아졌는지 확인
os.listdir('.')

['.config',
 'WikiExtractor.py',
 'kowiki-latest-pages-meta-current.xml.bz2',
 'sample_data',
 'sdrive',
 '.ipynb_checkpoints',
 'udrive']

In [71]:
# WikiExtractor 실행
os.system(f"python WikiExtractor.py -o kowiki --json kowiki-latest-pages-meta-current.xml.bz2")

0

In [73]:
with open(os.path.join('kowiki', 'AA', 'wiki_00')) as f:
  for i, line in enumerate(f):
    if i >= 10:
      break
    line = line.strip()
    print(line)

{"id": "5", "url": "https://ko.wikipedia.org/wiki?curid=5", "title": "지미 카터", "text": "지미 카터\n\n제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.\n\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 \"땅콩 농부\" (Peanut Farmer)로 알려졌다.\n\n1962년 조지아주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받고 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.\n\n임기 말, 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 

In [74]:
filepaths = []
dirnames = os.listdir('kowiki')
for dirname in dirnames:
  dirpath = os.path.join('kowiki', dirname)
  filenames = os.listdir(dirpath)
  for filename in filenames:
    if filename.startswith('wiki_'):
      filepath = os.path.join(dirpath, filename)
      filepaths.append(filepath)
filepaths = sorted(filepaths)
print(len(filepaths), filepaths[:10])

810 ['kowiki/AA/wiki_00', 'kowiki/AA/wiki_01', 'kowiki/AA/wiki_02', 'kowiki/AA/wiki_03', 'kowiki/AA/wiki_04', 'kowiki/AA/wiki_05', 'kowiki/AA/wiki_06', 'kowiki/AA/wiki_07', 'kowiki/AA/wiki_08', 'kowiki/AA/wiki_09']


In [76]:
def trim_text(item):
  """
  한 위키 문서 내의 여러줄띄키(\n\n...)를 한 줄 띄기(\n)로 변경
  :param item: 위키 항목
  :return: text의 여러줄 new line을 한 줄 new line으로 변경한 json data
  """
  data = json.loads(item)
  text = data["text"]
  value = list(filter(lambda x: len(x) > 0, text.split('\n')))
  data["text"] = "\n".join(value)
  return data

In [77]:
# 여러 줄 띄기(\n\n...)를 한 줄 띄기(\n)로 변경
dataset = []
for filepath in tqdm(filepaths):
  with open(filepath, 'r') as f:
    for line in f:
      line = line.strip()    # 양쪽에 공백이 있으면 자름
      if line:
        dataset.append(trim_text(line))
print(len(dataset))

  0%|          | 0/810 [00:00<?, ?it/s]

584607


In [78]:
# 위키를 한 파일로 저장
with open('kowiki.txt', 'w') as f:
  for data in tqdm(dataset):
    f.write(data["text"])
    f.write('\n\n\n')

  0%|          | 0/584607 [00:00<?, ?it/s]

In [79]:
# 파일 내용 확인
with open("kowiki.txt") as f:
  for i, line in enumerate(f):
    if i >= 30:
      break
    line = line.strip()
    print(line)

지미 카터
제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.
조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
1962년 조지아주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.
1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받고 제럴드 포드 대통령을 누르고 당선되었다.
카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.
카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.
소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.
카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.
임기 말, 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 하였다.
그는 주이란 미국 대사관 인질 사건의 인질 구출 실패로 인한 원인으로, 1980년 제40대 대통령 선거에서 공화당의 로널드 레이건에게 패하며 재선에 실패하였다.
대한민국과의 관계
지미

In [80]:
# 압축
!zip kowiki.txt.zip kowiki.txt

  adding: kowiki.txt (deflated 64%)


In [81]:
# 압축파일 보관
shutil.move("kowiki.txt.zip", os.path.join(kowiki_dir, 'kowiki.txt.zip'))
os.listdir(kowiki_dir)

['kowiki.txt.zip']

▷ sentencepiece vocabulary 생성
  
특수 Token : 7개 정의  
vocabulary : 32,000개  
학습방법 : unigram  
  
특수 Token들  
● [PAD]: 자연어처리 문장들의 길이가 다르기 때문에 길이를 맞춰주기 위해 추가하는 token

● [UNK]: vocabulary에 등록되지 않는 단어가 있는 경우 [UNK]로 처리함

● [BOS]: Begin of Sequence로 문장의 시작을 의미하는 token

● [EOS]: End of Sequence로 문장의 끝을 의미하는 token

● [SEP]: Separator로 두 개 이상의 문장을 구분하기 위한 token

● [CLS]: Class로 문장을 대표하는 벡터를 의미하는 token

● [MASK]: Mask로 단어를 가리고자 할 때 사용하는 token

In [104]:
# sentencepiece 학습 함수
def trian_sentencepiece(corpus, prefix, vocab_size=32000):
  """
  sentencepiece를 이용해 vocab 학습
  :param corpus: 학습할 말뭉치
  :param prefix: 저장할 vocab 이름
  :param vocab_size: vocab 개수
  """
  spm.SentencePieceTrainer.train(
      f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size+7}" +
      # 7은 특수문자 개수
      " --model_type=unigram" +
      " --max_sentence_length=999999" + # 문장 최대 길이
      " --pad_id=0 --pad_piece=[PAD]" + # pad 지정
      " --unk_id=1 --unk_piece=[UNL]" + # unknown token 지정
      " --bos_id=2 --bos_piece=[BOS]" + # bos 지정
      " --eos_id=3 --eos_piece=[EOS]" + # eos 지정
      " --user_defined_symbols=[SEP], [CLS], [MASK]")

In [106]:
# sentencepiece 생성
# vocab 생성
train_sentencepiece('kowiki.txt', 'ko_32000', vocab_size=32000)

In [107]:
# vocabulary 파일 보관
# 생성된 vocab 이동
shutil.move('ko_32000.model', os.path.join(data_dir, 'ko_32000.model'))
shutil.move('ko_32000.vocab', os.path.join(data_dir, 'ko_32000.vocab'))
os.listdir(data_dir)

['kowiki', 'ko_32000.model', 'ko_32000.vocab']

▷ sentencepiece vocabulary 확인

In [108]:
# ko_32000.model 로드
# load vocab
vocab = spm.SentencePieceProcessor()
vocab.load(os.path.join(data_dir, 'ko_32000.model'))

True

In [109]:
# vocab 출력
print(f'len: {len(vocab)}')
for _id in range(16):
  print(f'{_id:5d}: {vocab.id_to_piece(_id)}')

len: 32007
    0: [PAD]
    1: [UNK]
    2: [BOS]
    3: [EOS]
    4: [SEP]
    5: [CLS]
    6: [MASK]
    7: .
    8: ,
    9: 의
   10: 는
   11: ▁
   12: )
   13: 에
   14: 년
   15: (


In [111]:
text =  '위키백과의 최상위 도메인이 .com이던 시절 ko.wikipedia.com에 구판 미디어위키가 깔렸으나 한글 처리에 문제가 있어 글을 올릴 수도 없는 이름뿐인 곳이었다.'

pieces = vocab.encode_as_pieces(text)
print(pieces[0:100])

['▁위키백과', '의', '▁최상위', '▁도메인', '이', '▁', '.', 'com', '이던', '▁시절', '▁', 'ko', '.', 'w', 'iki', 'pe', 'dia', '.', 'com', '에', '▁구', '판', '▁미디어', '위', '키', '가', '▁깔', '렸으나', '▁한글', '▁처리', '에', '▁문제가', '▁있어', '▁글을', '▁올릴', '▁수도', '▁없는', '▁이름', '뿐', '인', '▁곳이었다', '.']


In [112]:
print(vocab.decode_pieces(pieces))

위키백과의 최상위 도메인이 .com이던 시절 ko.wikipedia.com에 구판 미디어위키가 깔렸으나 한글 처리에 문제가 있어 글을 올릴 수도 없는 이름뿐인 곳이었다.


In [113]:
piece_ids = []
for piece in pieces:
  piece_ids.append(vocab.piece_to_id(piece))
print(piece_ids[0:100])

[11058, 9, 10085, 7498, 17, 11, 7, 4120, 3278, 1181, 11, 10205, 7, 1747, 27003, 4468, 14908, 7, 4120, 13, 221, 557, 2677, 212, 322, 19, 13354, 11054, 4747, 1599, 13, 1939, 509, 3828, 23300, 606, 495, 1553, 4478, 31, 21679, 7]


In [117]:
# text를 id로 tokenize 함
# sentence to ids
ids = vocab.encode_as_ids(text)
print(ids[0:100])

[11058, 9, 10085, 7498, 17, 11, 7, 4120, 3278, 1181, 11, 10205, 7, 1747, 27003, 4468, 14908, 7, 4120, 13, 221, 557, 2677, 212, 322, 19, 13354, 11054, 4747, 1599, 13, 1939, 509, 3828, 23300, 606, 495, 1553, 4478, 31, 21679, 7]


In [118]:
print(vocab.decode_ids(ids))

위키백과의 최상위 도메인이 .com이던 시절 ko.wikipedia.com에 구판 미디어위키가 깔렸으나 한글 처리에 문제가 있어 글을 올릴 수도 없는 이름뿐인 곳이었다.


In [120]:
id_pieces = []
for id in ids:
 id_pieces.append(vocab.id_to_piece(id))
print(id_pieces[0:100])

['▁위키백과', '의', '▁최상위', '▁도메인', '이', '▁', '.', 'com', '이던', '▁시절', '▁', 'ko', '.', 'w', 'iki', 'pe', 'dia', '.', 'com', '에', '▁구', '판', '▁미디어', '위', '키', '가', '▁깔', '렸으나', '▁한글', '▁처리', '에', '▁문제가', '▁있어', '▁글을', '▁올릴', '▁수도', '▁없는', '▁이름', '뿐', '인', '▁곳이었다', '.']
